In [11]:
!pip install konlpy gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 29.5 MB/s eta 0:00:00


In [4]:
import pandas as pd

In [13]:
product_info_data = pd.read_csv("product_info.csv")
sales_data = pd.read_csv("sales.csv")
train_data = pd.read_csv("train.csv")

FileNotFoundError: ignored

In [ ]:
print(len(product_info_data),len(sales_data))

12778 15890


In [ ]:
product_info_data.head()

,제품,제품특성
0,B002-03509-00001,제품유형:일반식품 콜라겐 펩타이드:1000mg 종류:어류 분자량:300Da 섭취대상...
1,B002-02376-00001,700mg x 28정
2,B002-03735-00001,제품타입:정 섭취방법:물과 함께 섭취대상:성인남녀 섭취횟수:하루 두 번 1일 총 섭...
3,B002-03735-00003,제품타입:정 섭취방법:물과 함께 섭취대상:성인남녀 섭취횟수:하루 한 번 1일 총 섭...
4,B002-02769-00001,HCA:900mg 영양소 원료명(식약처고시):비타민B1 영양소 원료명(식약처고시):...


In [ ]:
product_cate_data = pd.merge(left = product_info_data , right = sales_data[['제품','소분류']], how = "inner", on = "제품")

In [ ]:
#소분류 기준으로 제품 특성 합치기
detail_cate_info_data = product_cate_data.groupby('소분류')['제품특성'].apply(lambda x: ','.join(x)).reset_index()
detail_cate_info_data.head()

,소분류,제품특성
0,B002-C003-0001,제품유형:일반식품 콜라겐 펩타이드:1000mg 종류:어류 분자량:300Da 섭취대상...
1,B002-C003-0002,비타민D:25µg 1일 총 섭취량:1정 제품용량:2개월분 제품타입:정 섭취횟수:하루...
2,B002-C003-0003,종류_A섭취대상:아르기닌 섭취방법:성인남녀 1일 총 섭취량:물과 함께 섭취횟수:2정...
3,B002-C003-0004,단백질종류:동물성+식물성 단백질성분:농축유청단백질(WPC) 단백질성분:분리대두단백질...
4,B002-C003-0005,홍삼농축액 함량:8% 고형분 함량:60% 포장형태:스틱형 1일 총 섭취량:1포 제품...


In [ ]:
import pandas as pd
from konlpy.tag import Okt
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

# 형태소 분석기 초기화
okt = Okt()

# 텍스트를 형태소로 분리
texts = [okt.nouns(line) for line in detail_cate_info_data['제품특성']]

# 사전 생성
dictionary = Dictionary(texts)

# 코퍼스 생성
corpus = [dictionary.doc2bow(text) for text in texts]

# TF-IDF 모델 생성
tfidf_model = TfidfModel(corpus)

In [ ]:
# 각 행에서 상위 키워드 출력
for i, text in enumerate(texts):
    print(f"Row {i} Keywords:")
    tfidf_values = tfidf_model[corpus[i]]
    sorted_tfidf_values = sorted(tfidf_values, key=lambda x: x[1], reverse=True)
    for word_id, score in sorted_tfidf_values[:500]:  # 상위 100개 키워드 출력
        #print(dictionary[word_id], score)
        keyword = dictionary[word_id]
        if keyword in detail_cate_info_data.loc[i, '제품특성']:
            # 해당 키워드 이름으로 새로운 컬럼 생성, 값은 True로 설정
            detail_cate_info_data.loc[i, keyword] = True

# NaN값을 False로 채움
detail_cate_info_data.fillna(False, inplace=True)



In [ ]:
# 채워지지 않은 키워드 행들 False로 채우기

In [ ]:
detail_cate_info_data.head()

In [ ]:
print(detail_cate_info_data.nunique())

소분류     53
제품특성    53
섭취       2
식약처      2
영양소      2
        ..
된장       2
수확       2
절단       2
중후       2
호두       2
Length: 1390, dtype: int64


In [ ]:
detail_cate_info_data.to_csv("detail_cate_keyword.csv",index = False)

In [5]:
detail_cate_info_data = pd.read_csv("detail_cate_keyword.csv")

In [6]:
detail_cate_info_data.head()

,Unnamed: 0,소분류,제품특성,섭취,식약처,영양소,고시,원료,성인,단백질,...,고등어,국수,농산물,농약,닭백숙,된장,수확,절단,중후,호두
0,0,B002-C003-0001,제품유형:일반식품 콜라겐 펩타이드:1000mg 종류:어류 분자량:300Da 섭취대상...,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,1,B002-C003-0002,비타민D:25µg 1일 총 섭취량:1정 제품용량:2개월분 제품타입:정 섭취횟수:하루...,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,2,B002-C003-0003,종류_A섭취대상:아르기닌 섭취방법:성인남녀 1일 총 섭취량:물과 함께 섭취횟수:2정...,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3,3,B002-C003-0004,단백질종류:동물성+식물성 단백질성분:농축유청단백질(WPC) 단백질성분:분리대두단백질...,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
4,4,B002-C003-0005,홍삼농축액 함량:8% 고형분 함량:60% 포장형태:스틱형 1일 총 섭취량:1포 제품...,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False


In [19]:
drop_cate_list = []
for c in detail_cate_info_data.columns[3:]:
    cate_count = list(detail_cate_info_data[c].value_counts())
    if cate_count[1] < 10:
        drop_cate_list.append(c)

In [21]:
detail_cate_info_data.drop(columns=drop_cate_list)

,Unnamed: 0,소분류,제품특성,섭취,원료,총,방법,횟수,대상,정,...,효과,케어,제거,각질,오일,단계,진정,로션,강화,샴푸
0,0,B002-C003-0001,제품유형:일반식품 콜라겐 펩타이드:1000mg 종류:어류 분자량:300Da 섭취대상...,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,1,B002-C003-0002,비타민D:25µg 1일 총 섭취량:1정 제품용량:2개월분 제품타입:정 섭취횟수:하루...,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,2,B002-C003-0003,종류_A섭취대상:아르기닌 섭취방법:성인남녀 1일 총 섭취량:물과 함께 섭취횟수:2정...,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3,3,B002-C003-0004,단백질종류:동물성+식물성 단백질성분:농축유청단백질(WPC) 단백질성분:분리대두단백질...,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
4,4,B002-C003-0005,홍삼농축액 함량:8% 고형분 함량:60% 포장형태:스틱형 1일 총 섭취량:1포 제품...,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
5,5,B002-C003-0006,"형태:액체형 고정방식:비치형 :1개, 형태:액체형 고정방식:비치형 :1개, 형태:액...",False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,6,B002-C003-0007,"형태:고체형, 형태특징사용장소사용장소사용장소:고체형, 영양소 원료명(식약처고시)제품...",True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
7,7,B002-C003-0008,"구성:본품 용량:5.5L 세탁기유형:일반,드럼겸용 성분:무미세플라스틱 성분:무방부제...",False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
8,8,B002-C003-0009,"사용횟수:일회용 재질:스테인리스스틸, 사용횟수:일회용 재질:스테인리스스틸, 사용횟수...",False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
9,9,B002-C003-0010,"사이즈:중 :1개 40매 56339,가로사이즈:17cm :1개, 가로사이즈:25cm...",False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [20]:
print(drop_cate_list)

['식약처', '영양소', '고시', '성인', '단백질', '량', '하루', '남녀', '번', '비타민', '주요', '물', '포', '체지방', '감소', '유청', '어류', '분자량', '분말', '두', '맛', '건강', '음용', '스푼', '식이섬유', '바로', '비오틴', '펩타이드', '판토텐산', '분리', '캡슐', '초코', '콜라겐', '동물성', '식물성', '카테킨', '나이아신', '조리', '셀레늄', '셀렌', '해당', '아연', '구미', '젤리', '웨이', '보관', '농축', '열량', '닉산', '후코', '개인', '가수분해', '전자레인지', '장', '엽산', '잔틴', '실온', '별', '공액', '리놀레산', '밥', '망간', '상주', '중량', '우유', '영양성분', '잡곡', '요오드', '히알루론산', '무게', '철', '품질', '칼슘', '백미', '구리', '마그네슘', '카페인', '곡물', '보충', '준치', '충족', '수', '크롬', '몰리브덴', '뼈', '관절', '츄어블', '청소년', '여성', '일', '항산화', '베타카로틴', '남성', '혈행', '권장', '임산부', '루테', '생균', '칼륨', '면역', '눈', '시니어', '플라보노이드', '피로', '회복', '지아', '간', '실리', '마린', '만', '철분', '함량', '방울', '질', '코엔자임', '전립선', '로르산', '산', '아르기닌', '재료', '대두', '홍삼', '기억', '산양', '유', '딸기', '아미노산', '게이', '쿠키', '카제인', '바닐라', '너', '커피', '무맛', '바나나', '복숭아', '매스', '레몬', '색상', '노즐', '온도', '널', '글루타민', '크레아틴', '틴', '퍼스', '드라이버', '프로', '난백', '부스터', '농축액', '고형', '사이드', '스틱', '추출', '파우치', '앰플형', '갱년기',

In [23]:
detail_cate_info_data.to_csv("detail_cate_keyword.csv",index = False)

In [24]:
detail_cate_info_data = pd.read_csv("detail_cate_keyword.csv")

In [25]:
detail_cate_info_data.head()

,Unnamed: 0,소분류,제품특성,섭취,식약처,영양소,고시,원료,성인,단백질,...,고등어,국수,농산물,농약,닭백숙,된장,수확,절단,중후,호두
0,0,B002-C003-0001,제품유형:일반식품 콜라겐 펩타이드:1000mg 종류:어류 분자량:300Da 섭취대상...,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,1,B002-C003-0002,비타민D:25µg 1일 총 섭취량:1정 제품용량:2개월분 제품타입:정 섭취횟수:하루...,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,2,B002-C003-0003,종류_A섭취대상:아르기닌 섭취방법:성인남녀 1일 총 섭취량:물과 함께 섭취횟수:2정...,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3,3,B002-C003-0004,단백질종류:동물성+식물성 단백질성분:농축유청단백질(WPC) 단백질성분:분리대두단백질...,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
4,4,B002-C003-0005,홍삼농축액 함량:8% 고형분 함량:60% 포장형태:스틱형 1일 총 섭취량:1포 제품...,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
